In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


## Setup training/test data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train_l), (x_test, y_test_l) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_l, num_classes)
y_test = keras.utils.to_categorical(y_test_l, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Load pretrained model

In [3]:
model_save_name = 'mnist_classifier'
path = F"{model_save_name}" 
#model.save(path)
model = keras.models.load_model(path)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

## Compute network layer activations for each image

In [4]:
import nap
nap = nap.NeuralActivationPattern(x_test[:1000], y_test_l[:1000], model)
nap.summary(0)

             Layer    Activation shape
0           conv2d  (1000, 26, 26, 32)
1    max_pooling2d  (1000, 13, 13, 32)
2         conv2d_1  (1000, 11, 11, 64)
3  max_pooling2d_1    (1000, 5, 5, 64)
4          flatten        (1000, 1600)
5          dropout        (1000, 1600)
6            dense          (1000, 10)
Layer 0, number of clusters: 2
Layer 1, number of clusters: 2
Layer 2, number of clusters: 2
Layer 3, number of clusters: 13
Layer 4, number of clusters: 10
Layer 5, number of clusters: 10
Layer 6, number of clusters: 15


In [5]:
nap.summary(5)

## Show the images generating highest activations for each layer

In [86]:
def show_image_grid(images, labels, img_idx, title, images_per_row, img_scale):
    import plotly.express as px
    imSize = (images[0].shape[0], images[0].shape[1])
    # Get indices of the largest 
    sampleSize = len(img_idx)
    nCols = min(sampleSize, images_per_row)
    nRows =  max(1, int(sampleSize / images_per_row))
    img = np.array([images[idx].reshape(imSize) for idx in img_idx])
    fig = px.imshow(img, facet_col=0, binary_string=True, facet_col_wrap=nCols, 
                    #width=int(nCols*imSize[0]*img_scale), 
                    # height=220+int(nRows*80 + nRows*imSize[1]*img_scale), 
                    title=title).update_layout(margin=dict(l=5, b=0))
    # Set facet titles
    for i, im in enumerate(img_idx):
        fig.layout.annotations[i]['text'] = f"{labels[im]}"
    fig.show()

def show_cluster(average, representatives, outliers, images, labels, title):
    import plotly.express as px
    import plotly.subplots as sp
    imSize = (images[0].shape[0], images[0].shape[1])
    img = np.array([average.reshape(imSize)] + [images[idx].reshape(imSize) for idx in representatives] + [images[idx].reshape(imSize) for idx in outliers])
    fig = px.imshow(img, facet_col=0, binary_string=True, title = title)
    fig.layout.annotations[0]['text'] = "Average"
    for i, im in enumerate(representatives):
        fig.layout.annotations[1+i]['text'] = f"Representative | {labels[im]}"
    fig.layout.annotations[1]['text'] = "Representative"
    for i, im in enumerate(outliers):
        fig.layout.annotations[len(representatives)+i]['text'] = f"Outlier | {labels[im]}"
    fig.show()

def show_images(images, labels, layer_img_idx, titles, images_per_row = 10, img_scale = 2.0):
    for layer, img_idx in enumerate(layer_img_idx):
        show_image_grid(images, labels, img_idx, titles[layer], images_per_row, img_scale)

In [7]:
max_activations = [nap.get_max_activations(layerId, agg_func = np.average) for layerId in range(len(model.layers))]
titles = [f"Highest activating images for layer {layerId}" for layerId in range(len(max_activations))]
show_images(x_test, y_test_l, max_activations, titles)

# Neural Activation Clustering

In [87]:
nSamplesPerLayer = 10
layerId = 3

## Show inputs treated the same way by the network

sorted_clusters = nap.sorted(layerId)

for cluster_id, cluster in cluster_samples.groupby('clusterId'):
    avg = nap.average(cluster.index)
    centers = cluster.head(1)
    outliers = cluster.tail(3)
    show_cluster(avg, centers, outliers, x_test, y_test_l, F"Layer {layerId}, Cluster: {cluster_id}, Size: {len(cluster)}")

In [9]:

frac = 0.1
cluster_samples = nap.head(layerId, nSamplesPerLayer) 
titles = []
clusters = []
for cluster_id, cluster in cluster_samples.groupby('clusterId'):
    clusters.append(cluster.index)
    titles.append(F"Layer {layerId}, Cluster: {cluster_id} - Images most likely belonging to cluster")
show_images(x_test, y_test_l, clusters, titles)

### Show aggregated inputs treated the same way 

In [14]:
for layerId, layer in enumerate(model.layers):
    averages, clusterIds, cluster_sizes = nap.averages(layerId)
    cluster_labels = [f"Id: {clusterId}, Size: {size}" for clusterId, size in zip(clusterIds, cluster_sizes)]
    nClusters = len(averages)
    indices = list(range(nClusters))
    show_images(averages, cluster_labels, [indices], [f"Layer {layerId}, Clusters: {nClusters} - Cluster averages"])

In [ ]:
for layerId, layer in enumerate(model.layers):
    clusters, clusterIds, cluster_sizes = nap.get_clusters(layerId)
    # representative = [representative[0] for representative in pattern_representatives]
    # outliers =  [representative[-1] for representative in pattern_representatives]
    cluster_representative = [[cluster[0]] + cluster[-min(len(cluster), 3+1):] for cluster in clusters]
    # pattern_representative = representative + outliers
    cluster_labels = [f"Cluster {clusterId}" for clusterId in clusterIds]
    nClusters = len(clusters)
    titles = [F"Cluster: {cluster_id}, size: {cluster_size}" for cluster_id, cluster_size in zip(clusterIds, cluster_sizes)]
    print(F"Layer {layerId}")
    show_images(x_test, y_test_l, cluster_representative, titles, img_scale=0.1)
    


### Show activation for each feature. Warning!!! slow code. 

In [ ]:

# Convert to slow, but plotting friendly pandas DataFrame
activations_df = {'input_data': [], 'layer': [], 'feature': [], 'activation':[]}
for dp in range(len(layer_activations)):
  for feature in range(len(nap.layer_activations[0])):
      for activation in layer_activations[:][feature]:
        activations_df['input_data'].append(dp)
        activations_df['layer'].append(clusterOnLayer)
        activations_df['feature'].append(feature)
        activations_df['activation'].append(activation)

activations_df = pd.DataFrame(activations_df)
activations_df.describe()

In [ ]:
px.scatter(activations_df, x='feature', y='activation')